In [ ]:
## get initial set of labels from karma3

In [1]:
import pandas as pd
from google.cloud import bigquery
import pandas as pd
import glob
client = bigquery.Client()

savepath = f"data"

In [3]:
import requests

response = requests.get("https://lens-api.k3l.io/profile_count?strategy=creator")
count = response.json()['count']

response = requests.get(f"https://lens-api.k3l.io/profile_scores?strategy=creator&limit={count}")
karma_data = response.json()
karma_data = pd.DataFrame(karma_data)
karma_data.to_csv('data/karma_data.csv')
del response
karma_data.head()

,score,rank,handle,followersCount,id
0,0.064284,1,stani.lens,57607,5
1,0.059850,2,bradorbradley.lens,33082,36
2,0.059520,3,christina.lens,28154,142
3,0.055198,4,blackdave.lens,819,36284
4,0.054165,5,chriscomrie.lens,4917,73557


In [4]:
karma_data.shape

(100004, 5)

In [5]:
## get bigquery data

In [13]:


def dispatcher_data_query():
    return """
SELECT
  profile_id,
  profile_dispatcher_address,
  block_timestamp AS dispatcher_setup_timestamp
FROM
  `lens-public-data.polygon.public_profile_dispatcher`
"""




# Perform a query.
QUERY = dispatcher_data_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
df.to_csv(f'{savepath}/dispatcher_data')
df.shape

(82988, 3)

In [11]:

def aggregated_stats_query():
    return f"""
SELECT
  profile_id,
  total_posts,
  total_comments,
  total_mirrors,
  total_collects,
  datastream_metadata.source_timestamp
FROM
  `lens-public-data.polygon.public_profile_stats`
"""

# Perform a query.
QUERY = aggregated_stats_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
df.to_csv(f'{savepath}/aggregated_stats')
df.shape

(116483, 6)

In [12]:
def ens_address_query():
    return f"""
SELECT
  DISTINCT address
FROM
  `lens-public-data.polygon.ens_address_reverse_record`
"""

# Perform a query.
QUERY = ens_address_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
df.to_csv(f'{savepath}/ens_address')
df.shape

(47508, 1)

In [19]:
def proof_of_humanity_query():
    return f"""
SELECT
  DISTINCT address
FROM
  `lens-public-data.polygon.proof_of_humanity_address_to_proof_record`
WHERE
  is_registered=true
"""

# Perform a query.
QUERY = proof_of_humanity_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
df.to_csv(f'{savepath}/proof_of_humanity')
df.shape

(855, 1)

In [20]:
def sybil_dot_org_address_verified_twitter_query():
    return f"""
SELECT
  DISTINCT address
FROM
  `lens-public-data.polygon.sybil_dot_org_address_verified_twitter`
"""

# Perform a query.
QUERY = sybil_dot_org_address_verified_twitter_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
df.to_csv(f'{savepath}/sybil_dot_org_address_verified_twitter')
df.shape

(2784, 1)

In [22]:
def worldcoin_human_check_query():
    return f"""
SELECT
  profile_id
FROM
  `lens-public-data.polygon.worldcoin_human_check`
WHERE
  verified=true
"""

# Perform a query.
QUERY = worldcoin_human_check_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
df.to_csv(f'{savepath}/worldcoin_human_check')
df.shape

(54, 1)

In [24]:
def most_used_app_query():
    return f"""
SELECT
  profile_id,
  ARRAY_AGG( STRUCT(app_id,
      total_posts)
  ORDER BY
    total_posts DESC
  LIMIT
    1 )[
OFFSET
  (0)].app_id AS app_id_highest_posts,
  ARRAY_AGG( STRUCT(app_id,
      total_posts)
  ORDER BY
    total_comments DESC
  LIMIT
    1 )[
OFFSET
  (0)].app_id AS app_id_highest_comments,
FROM
  `lens-public-data.polygon.public_app_profile_stats`
GROUP BY
  profile_id
"""

# Perform a query.
QUERY = most_used_app_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
df.to_csv(f'{savepath}/most_used_app')
df.shape

(94732, 3)

In [2]:
import numpy as np
def q90(x):
    return x.quantile(0.9)
def q10(x):
    return x.quantile(0.1)
def q25(x):
    return x.quantile(0.25)
def q75(x):
    return x.quantile(0.75)


In [15]:
def getlags(df, lags=-1, by='profile_id'):
    df = df.sort_values(by=[by,'block_timestamp'],ascending=[True,False])
    shifted = df.groupby(by).shift(lags)
    df1 = df.join(shifted.rename(columns=lambda x: x+"_lag"))
    df1['diff'] = (df1['block_timestamp'] - df1['block_timestamp_lag']).dt.total_seconds()
    df1 = df1.groupby(by).agg({"diff": [np.mean, np.median,q10,q25,q75,q90,'max','min']})
    df1.columns = [f'{i}_{str(lags)[1:]}' for i in df1.columns]
    return df1

(84118, 24)

In [4]:
def like_data_insights_query():
    return f"""
SELECT
actioned_by_profile_id as profile_id, action_at as block_timestamp
FROM
  `lens-public-data.polygon.public_publication_reaction_records`
"""

# Perform a query.
QUERY = like_data_insights_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
print('running')
by = 'profile_id'
df1 = getlags(df, lags=-1, by=by)
df2 = getlags(df, lags=-10, by=by)
df3 = getlags(df, lags=-20, by=by)
df1 = df1.join(df2,on=by).join(df3,on=by)

df1.to_csv(f'{savepath}/like_data_insights')
df1.shape

running


TypeError: 'int' object is not subscriptable

In [7]:
def post_data_insights_query():
    return f"""
SELECT
  profile_id,
  block_timestamp
FROM
  `lens-public-data.polygon.public_profile_post`
WHERE
  s3_metadata_location IS NOT NULL
  AND has_error IS FALSE
  AND is_metadata_processed IS TRUE
  AND is_related_to_post IS NULL
  AND is_related_to_comment IS null
"""

# Perform a query.
QUERY = post_data_insights_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
print('running')
by = 'profile_id'
df1 = getlags(df, lags=-1, by=by)
df2 = getlags(df, lags=-10, by=by)
df3 = getlags(df, lags=-20, by=by)
df1 = df1.join(df2,on=by).join(df3,on=by)

df1.to_csv(f'{savepath}/post_data_insights')
df1.shape

running


(88681, 24)

In [8]:
def mirror_data_insights_query():
    return f"""
SELECT
  profile_id,
  block_timestamp
FROM
  `lens-public-data.polygon.public_profile_post`
WHERE
  s3_metadata_location IS NOT NULL
  AND has_error IS FALSE
  AND is_metadata_processed IS TRUE
  AND (is_related_to_post IS NOT NULL
    OR is_related_to_comment IS NOT NULL)
"""

# Perform a query.
QUERY = mirror_data_insights_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
print('running')
by = 'profile_id'
df1 = getlags(df, lags=-1, by=by)
df2 = getlags(df, lags=-10, by=by)
df3 = getlags(df, lags=-20, by=by)
df1 = df1.join(df2,on=by).join(df3,on=by)

df1.to_csv(f'{savepath}/mirror_data_insights')
df1.shape

running


(74481, 24)

In [9]:
def comment_data_insights_query():
    return f"""
SELECT
  comment_by_profile_id AS profile_id,
  block_timestamp
FROM
  `lens-public-data.polygon.public_post_comment`
WHERE
  s3_metadata_location IS NOT NULL
  AND has_error IS FALSE
  AND is_metadata_processed IS true
"""

# Perform a query.
QUERY = comment_data_insights_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
print('running')
by = 'profile_id'
df1 = getlags(df, lags=-1, by=by)
df2 = getlags(df, lags=-10, by=by)
df3 = getlags(df, lags=-20, by=by)
df1 = df1.join(df2,on=by).join(df3,on=by)

df1.to_csv(f'{savepath}/comment_data_insights')
df1.shape

running


(72821, 24)

In [10]:
def collect_data_insights_query():
    return f"""
SELECT
  collected_by AS address,
  block_timestamp
FROM
  `lens-public-data.polygon.public_publication_collect_module_collected_records`
"""

# Perform a query.
QUERY = collect_data_insights_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
print('running')
by = 'address'
df1 = getlags(df, lags=-1, by=by)
df2 = getlags(df, lags=-10, by=by)
df3 = getlags(df, lags=-20, by=by)
df1 = df1.join(df2,on=by).join(df3,on=by)

df1.to_csv(f'{savepath}/collect_data_insights')
df1.shape

running


KeyError: 'profile_id'

In [18]:
def paid_collect_data_insights_query():
    return f"""
WITH
  paid_pubs AS (
  SELECT
    publication_id
  FROM
    `lens-public-data.polygon.public_publication_collect_module_details`
  WHERE
    amount IS NOT NULL )
SELECT
  collected_by AS address,
  block_timestamp
FROM
  `lens-public-data.polygon.public_publication_collect_module_collected_records`
WHERE
  publication_id IN (
  SELECT
    publication_id
  FROM
    paid_pubs)
"""

# Perform a query.
QUERY = paid_collect_data_insights_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()
print('running')
by = 'address'
df1 = getlags(df, lags=-1, by=by)
df2 = getlags(df, lags=-10, by=by)
df3 = getlags(df, lags=-20, by=by)
df1 = df1.join(df2,on=by).join(df3,on=by)

df1.to_csv(f'{savepath}/paid_collect_data_insights')
df1.shape

running


(47945, 24)

In [97]:
def address_query():
    return f"""
SELECT
profile_id,
owned_by as address,
is_default,
metadata_block_timestamp,
block_timestamp
FROM
  `lens-public-data.polygon.public_profile`
"""

# Perform a query.
QUERY = address_query()
query_job = client.query(QUERY)  # API request
# rows = query_job.result()  # Waits for query to finish
df = query_job.to_dataframe()

df.to_csv(f'{savepath}/address')
df.shape

(116487, 5)